## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,54.32,43,75,1.01,broken clouds
1,1,Kapaa,US,22.0752,-159.3190,84.52,76,75,5.99,broken clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,59.88,26,0,9.40,clear sky
3,3,Port Elizabeth,ZA,-33.9180,25.5701,46.69,61,40,3.44,scattered clouds
4,4,Port Hawkesbury,CA,45.6169,-61.3485,78.01,87,75,4.00,broken clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kapaa,US,22.0752,-159.3190,84.52,76,75,5.99,broken clouds
4,4,Port Hawkesbury,CA,45.6169,-61.3485,78.01,87,75,4.00,broken clouds
6,6,Puerto Ayora,EC,-0.7393,-90.3518,72.55,93,93,7.00,overcast clouds
7,7,Avarua,CK,-21.2078,-159.7750,78.85,73,20,20.71,light rain
8,8,Razole,IN,16.4833,81.8333,78.48,88,90,3.31,overcast clouds
11,11,Jamestown,US,42.0970,-79.2353,72.30,97,90,1.99,moderate rain
12,12,Panzhihua,CN,26.5851,101.7128,70.79,95,100,1.14,overcast clouds
17,17,Vaini,TO,-21.2000,-175.2000,78.96,89,40,9.22,scattered clouds
22,22,Souillac,MU,-20.5167,57.5167,71.37,88,75,6.91,light rain
24,24,Auki,SB,-8.7676,160.7034,82.56,76,45,6.73,scattered clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                328
City                   328
Country                328
Lat                    328
Lng                    328
Max Temp               328
Humidity               328
Cloudiness             328
Wind Speed             328
Current Description    328
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,84.52,broken clouds,22.0752,-159.3190,
4,Port Hawkesbury,CA,78.01,broken clouds,45.6169,-61.3485,
6,Puerto Ayora,EC,72.55,overcast clouds,-0.7393,-90.3518,
7,Avarua,CK,78.85,light rain,-21.2078,-159.7750,
8,Razole,IN,78.48,overcast clouds,16.4833,81.8333,
11,Jamestown,US,72.30,moderate rain,42.0970,-79.2353,
12,Panzhihua,CN,70.79,overcast clouds,26.5851,101.7128,
17,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,
22,Souillac,MU,71.37,light rain,-20.5167,57.5167,
24,Auki,SB,82.56,scattered clouds,-8.7676,160.7034,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
  
    
        

Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig